In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import pathlib

## Glove embeddings in Gensim Word2Vec format

In [ ]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors

In [ ]:
if False:
    glove_file = datapath(str(path_to_embeddings))
    tmp_file = datapath('/opt/data/anuj/other/quora/glove.840B.300d/word2vec_format.txt')
    # call glove2word2vec script
    from gensim.scripts.glove2word2vec import glove2word2vec
    glove2word2vec(glove_file, tmp_file)
    model = KeyedVectors.load_word2vec_format(tmp_file)

    # NOTE: this part takes 10-11 mintutes on a fast computer!
    embeddings = KeyedVectors.load_word2vec_format('/opt/data/anuj/other/quora/glove.840B.300d/word2vec_format.txt')

    with open('/opt/data/anuj/other/quora/glove.840B.300d/pickle.pkl', 'wb') as f:
        pickle.dump(embeddings, f)

In [ ]:
embeddings = KeyedVectors.load_word2vec_format('/opt/data/anuj/other/quora/glove.840B.300d/word2vec_format.txt')

## Load the data

In [ ]:
path_to_train = pathlib.Path('/opt/data/anuj/other/quora/train.csv')
path_to_test = pathlib.Path('/opt/data/anuj/other/quora/test.csv')

In [ ]:
%%time
df_train_all = pd.read_csv(str(path_to_train))
df_heldout = pd.read_csv(str(path_to_test))

## Clean up the text

In [ ]:
from tqdm import tqdm
tqdm.pandas()

In [ ]:
from preprocess import preprocessor

In [ ]:
df_train_all["question_text"] = df_train_all["question_text"].progress_apply(lambda x: replace_typical_misspell(clean_text(x)))
sentences = df_train_all["question_text"].apply(lambda x: x.split())
vocab = build_vocab(sentences)

In [ ]:
oov = check_coverage(vocab, embeddings)

## Split train into train-val-test

In [ ]:
%%time
# Random split
n = df_train_all.shape[0]
n_train = int(0.7 * n)
n_val = int(0.2 * n)

df_train_all = df_train_all.sample(frac=1, axis=0, random_state=555)
assert df_train_all.shape[0] == n

df_train = df_train_all.iloc[: n_train]
df_val = df_train_all.iloc[n_train: n_train + n_val]
df_test = df_train_all.iloc[n_train + n_val:]
assert df_train.shape[0] + df_val.shape[0] + df_test.shape[0] == n

path_to_base = path_to_train.parent / 'processed'
df_train.to_csv(str(path_to_base / 'train.csv'), index=False)
df_val.to_csv(str(path_to_base / 'val.csv'), index=False)
df_test.to_csv(str(path_to_base / 'test.csv'), index=False)

In [ ]:
df_train = pd.read_csv(str(path_to_base / 'train.csv'))

In [ ]:
df_train.head()